In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

There's a new jupysql version available (0.10.2), you're running 0.7.2. To upgrade: pip install jupysql --upgrade


# coalesceの使い所

## 条件が1つも当てはまらなかった時(=NULL)のデフォルト値をうめる
+ 空行の場合はsumはNULLを返すので、それの穴埋め
  + countの場合は空行の場合0を返す。

In [23]:
%%sql
drop table if exists PrinterControl;
CREATE TABLE PrinterControl
(user_id CHAR(10), -- NULLは空きプリンタを意味する
 printer_name CHAR(4) NOT NULL PRIMARY KEY,
 printer_description CHAR(40) NOT NULL);

INSERT INTO PrinterControl VALUES( 'chacha',  'LPT1',  '一階のプリンタ');
INSERT INTO PrinterControl VALUES( 'lee'   ,  'LPT2',  '二階のプリンタ'); 
INSERT INTO PrinterControl VALUES( 'thomas',  'LPT3',  '三階のプリンタ' );
INSERT INTO PrinterControl VALUES( NULL    ,  'LPT4',  '共有プリンタ' );
INSERT INTO PrinterControl VALUES( NULL    ,  'LPT5',  '共有プリンタ' );

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [25]:
%%sql
-- 空行の場合もmin(printer_name) -> NULLを返す
select coalesce(
        min(printer_name),
        (
            select min(printer_name)
            from PrinterControl
            where user_id is null
        )
    )
from PrinterControl
where user_id = 'dummy'

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


coalesce
LPT4


## すべての列がnullであるものを探す

In [21]:
%%sql
drop table if exists Arraytbl;
CREATE TABLE ArrayTbl
 (keycol CHAR(1) PRIMARY KEY,
  col1  INTEGER,
  col2  INTEGER,
  col3  INTEGER,
  col4  INTEGER,
  col5  INTEGER,
  col6  INTEGER,
  col7  INTEGER,
  col8  INTEGER,
  col9  INTEGER,
  col10 INTEGER);

--オールNULL
INSERT INTO ArrayTbl VALUES('A', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL);
INSERT INTO ArrayTbl VALUES('B', 3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.


[]

In [22]:
%%sql
SELECT *
  FROM ArrayTbl
 WHERE COALESCE(col1, col2, col3, col4, col5, col6, col7, col8, col9, col10) IS NULL;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


keycol,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10
A,None,None,None,None,None,None,None,None,None,None


## 現在行と次の行を比較して一括updateする
  + 最後の行はnullになるのでcoalesce使う

In [17]:
%%sql
drop table if exists seq;
create table seq (
    id integer primary key,
    value integer not null,
    diff integer not null
);
insert into seq
values (1,1,0),
    (2,3,0),
    (3,4,0),
    (4,6,0),
    (5,7,0),
    (6,8,0),
    (7,9,0);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
7 rows affected.


[]

In [20]:
%%sql
update seq as s1
set diff = coalesce(
    (
        select min(value)
        from seq as s2
        where s1.value < s2.value
    ) - s1.value
, 0);

select *
from seq;

*  postgresql://padawan:***@db:5432/dsdojo_db
7 rows affected.
7 rows affected.


id,value,diff
1,1,2
2,3,1
3,4,2
4,6,1
5,7,1
6,8,1
7,9,0


## マスターデータの中で実績のない行を列挙する
+ 顧客(Customer_id)の中で売上実績(Order)がないものなど

In [27]:
%%sql
drop table if exists orders, customers;
create table customers (
    id integer primary key,
    name varchar(32) not null
);

create table orders (
    customer_id integer references Customers(id),
    product_name varchar(32),
    amount integer not null check (amount > 0),
    primary key(customer_id, product_name)
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.


[]

In [28]:
%%sql
insert into customers
values (1, 'AAA'),
    (2, 'BBB'),
    (3, 'CCC'),
    (4, 'DDD');

insert into orders
values (1, 'zz', 2),
    (1, 'yy', 3),
    (3, 'xx', 1);

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.
3 rows affected.


[]

In [31]:
%%sql
-- left join + where is nullで対応
select distinct id
from customers as c
left join orders as o
on c.id = o.customer_id
where customer_id is null

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


id
4
2


## 1行同士をjoinする
+ 問題自体は各カラムについて初めて登場する(下から上に走査)非nullを見つけて、テーブル表示する

In [3]:
%%sql
drop table if exists Foobar;
CREATE TABLE Foobar
(lvl INTEGER NOT NULL PRIMARY KEY,
 color VARCHAR(10),
 length INTEGER,
 width INTEGER,
 hgt INTEGER);

INSERT INTO Foobar VALUES (1, 'RED', 8, 10, 12);
INSERT INTO Foobar VALUES (2, NULL, NULL, NULL, 20);
INSERT INTO Foobar VALUES (3, NULL, 9, 82, 25);
INSERT INTO Foobar VALUES (4, 'BLUE', NULL, 67, NULL);
INSERT INTO Foobar VALUES (5, 'GRAY', NULL, NULL, NULL);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [4]:
%%sql
select
    coalesce(f5.color, f4.color, f3.color, f2.color, f1.color) as color,
    coalesce(f5.length, f4.length, f3.length, f2.length, f1.length) as length,
    coalesce(f5.width, f4.width, f3.width, f2.width, f1.width) as width,
    coalesce(f5.hgt, f4.hgt, f3.hgt, f2.hgt, f1.hgt) as hgt
from Foobar as f1, Foobar as f2, Foobar as f3, Foobar as f4, Foobar as f5
where f1.lvl = 1 and f2.lvl = 2 and f3.lvl = 3 and f4.lvl = 4 and f5.lvl = 5

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


color,length,width,hgt
GRAY,9,67,25
